In [1]:
# Importing the required libraries
import os
from dotenv import load_dotenv
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Accessing the secrets from the environment variables
load_dotenv()
HF_Token = os.getenv("HF_TOKEN")

In [3]:
# Upload the dataset and transform to dataframe
# Define the dataset path
dataset_path = "../8_Testing_Input_and_Output/App_Output_2b.csv"
print("Dataset Path:", dataset_path)

# Check if the file exists at the specified path
if not os.path.isfile(dataset_path):
    raise FileNotFoundError(f"Unable to find the file at {dataset_path}")

# Load the dataset
testing_output_2b = load_dataset('csv', data_files=dataset_path)

# Convert the dataset to a pandas dataframe
df_2b_testing_output = testing_output_2b['train'].to_pandas()

# Print a few rows to verify
print(df_2b_testing_output.head())

Dataset Path: ../8_Testing_Input_and_Output/App_Output_2b.csv


Generating train split: 0 examples [00:00, ? examples/s]

            DB_ID                                              Query  \
0  concert_singer  SELECT T2.name ,  T2.capacity FROM concert AS ...   
1          pets_1  SELECT T1.fname ,  T1.age FROM student AS T1 J...   
2           car_1  SELECT T1.CountryName FROM COUNTRIES AS T1 JOI...   
3           car_1  SELECT T2.MakeId ,  T2.Make FROM CARS_DATA AS ...   
4           car_1  select t1.id ,  t1.maker from car_makers as t1...   

                                            Question  \
0  Show the stadium name and capacity with most n...   
1  Find the first name and age of students who ha...   
2  Which countries in europe have at least 3 car ...   
3  Among the cars with more than lowest horsepowe...   
4  Which are the car makers which produce at leas...   

                                              Output  \
0  **Translation:**\nSelect the name and capacity...   
1  **Translation:**\nSelect the first name and ag...   
2  **Translation:**  \nSelect the names of countr...   
3  **T

**Version 1**

In [4]:
# Load a pre-trained model for generating sentence embeddings
embedding_model_1 = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True) # https://huggingface.co/jinaai/jina-embeddings-v3

# Function to compute embeddings and similarity
def Translation_assessment_1(df_2b_testing_output):
    df_2b_testing_output['Question'] = df_2b_testing_output['Question'].fillna('').astype(str)
    df_2b_testing_output['Translation'] = df_2b_testing_output['Translation'].fillna('').astype(str)
    
    # Generate embeddings for the "Question" and "Translation" columns
    question_embeddings = embedding_model_1.encode(df_2b_testing_output['Question'].tolist(), task="text-matching", convert_to_tensor=True)
    translation_embeddings = embedding_model_1.encode(df_2b_testing_output['Translation'].tolist(), task="text-matching", convert_to_tensor=True)

    # Calculate cosine similarity for each row
    similarities = cosine_similarity(question_embeddings, translation_embeddings)

    # Since cosine_similarity returns a matrix, we extract the diagonal (row-wise comparison)
    df_2b_testing_output['Similarity_1'] = np.diagonal(similarities)

    return df_2b_testing_output

# Call the function and process the dataframe
df_translation_assessment_1 = Translation_assessment_1(df_2b_testing_output)

**Version 2**

In [5]:
# 1. Specify preferred dimensions
dimensions = 512

# 2. Load the model
embedding_model_2 = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)  # https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1

# Function to generate a detailed instruction for the query
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# Function to compute embeddings and similarity
def Translation_assessment_2(df_2b_testing_output):
    # Define task instruction for the queries
    task = 'Compare the question and translation to assess the quality of the translation.'

    # Add instruction to the "Question" column
    questions_with_instructions = [
        get_detailed_instruct(task, question) for question in df_2b_testing_output['Question'].tolist()
    ]

    # Generate a list of documents to encode
    docs = questions_with_instructions + df_2b_testing_output['Translation'].tolist()
    
    # 2. Encode
    embeddings = embedding_model_2.encode(docs)

    # Optional: Quantize the embeddings
    binary_embeddings = quantize_embeddings(embeddings, precision="ubinary")

    # Calculate cosine similarity between the first half (questions) and the second half (translations)
    question_embeddings = embeddings[:len(questions_with_instructions)]
    translation_embeddings = embeddings[len(questions_with_instructions):]

    # Calculate cosine similarity
    similarities = cos_sim(question_embeddings, translation_embeddings)

    # Since cos_sim returns a matrix, we extract the diagonal (row-wise comparison)
    df_2b_testing_output['Similarity_V2'] = np.diagonal(similarities.cpu().numpy())

    return df_2b_testing_output

# Call the function and process the dataframe
df_translation_assessment_2 = Translation_assessment_2(df_2b_testing_output)


**Version 3**

In [6]:
# Load a pre-trained model for generating sentence embeddings
embedding_model = SentenceTransformer("thenlper/gte-large")

# Function to compute embeddings and similarity
def Translation_assessment_3(df_2b_testing_output):
    # Generate embeddings for the "Question" and "Translation" columns
    question_embeddings = embedding_model.encode(df_2b_testing_output['Question'].tolist(), convert_to_tensor=True)
    translation_embeddings = embedding_model.encode(df_2b_testing_output['Translation'].tolist(), convert_to_tensor=True)

    # Calculate cosine similarity for each row
    similarities = cosine_similarity(question_embeddings, translation_embeddings)

    # Since cosine_similarity returns a matrix, we extract the diagonal (row-wise comparison)
    df_2b_testing_output['Similarity_V3'] = np.diagonal(similarities)

    return df_2b_testing_output

# Call the function and process the dataframe
df_translation_assessment_3 = Translation_assessment_3(df_2b_testing_output)

# Saving to CSV with the similarity score
df_translation_assessment_3.to_csv('../8_Testing_Input_and_Output/Translation_assessment_2b.csv', index=False)